In [1]:
import numpy as np
import tensorflow as tf
import t3f
tf.set_random_seed(0)
np.random.seed(0)
%matplotlib inline
import matplotlib.pyplot as plt
import metric_util as mt
import data_util as du
from t3f import shapes

/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Tensor completion

In this example we will see how can we do tensor completion with t3f, i.e. observe a fraction of values in a tensor and recover the rest by assuming that the original tensor has low TT-rank.
Mathematically it means that we have a binary mask $P$ and a ground truth tensor $A$, but we observe only a noisy and sparsified version of $A$: $P \odot (\hat{A})$, where $\odot$ is the elementwise product (applying the binary mask) and $\hat{A} = A + \text{noise}$. In this case our task reduces to the following optimization problem:
\begin{equation*}
\begin{aligned}
& \underset{X}{\text{minimize}} 
& & \|P \odot (X - \hat{A})\|_F^2 \\
& \text{subject to} 
& & \text{tt_rank}(X) \leq r_0
\end{aligned}
\end{equation*}



### Generating problem instance,
Lets generate a random matrix $A$, noise, and mask $P$.

In [2]:
subject_scan_path = du.get_full_path_subject1()
print "Subject Path: " + str(subject_scan_path)
x_true_org = mt.read_image_abs_path(subject_scan_path)

Subject Path: /work/pl/sch/analysis/data/COBRE001/swaAMAYER+cobre01_63001+M87100944+20110309at135133+RSTpre_V01_R01+CM.nii


In [3]:
x_true_img = np.array(x_true_org.get_data())

In [4]:
#shape = (3, 4, 4, 5, 7, 5)
shape = (53,63,46,144)
# Fix random seed so the results are comparable between runs.
tf.set_random_seed(0)
# Generate ground truth tensor A. To make sure that it has low TT-rank,
# let's generate a random tt-rank 5 tensor and apply t3f.full to it to convert to actual tensor.
#ground_truth = t3f.full(t3f.random_tensor(shape, tt_rank=5))
ground_truth = x_true_img
# Make a (non trainable) variable out of ground truth. Otherwise, it will be randomly regenerated on each sess.run.
ground_truth = tf.get_variable('ground_truth', initializer=ground_truth, trainable=False)
noise = 1e-2 * tf.get_variable('noise', initializer=tf.random_normal(shape), trainable=False)
noisy_ground_truth = ground_truth
# Observe 25% of the tensor values.
sparsity_mask = tf.cast(tf.random_uniform(shape) <= 0.60, tf.float32)
sparsity_mask = tf.get_variable('sparsity_mask', initializer=sparsity_mask, trainable=False)
sparse_observation = noisy_ground_truth * sparsity_mask

### Initialize the variable and compute the loss

In [5]:
def frobenius_norm_tf(x):
    return tf.reduce_sum(x ** 2) ** 0.5

In [6]:
def relative_error1(x_hat,x_true):
    percent_error = frobenius_norm_tf(x_hat - x_true) / frobenius_norm_tf(x_true)
    return percent_error

In [7]:
observed_total = tf.reduce_sum(sparsity_mask)
total = np.prod(shape)
ranks_a = np.array([53,63,46,144,1])
tt_with_ranks = t3f.to_tt_tensor(x_true_img, max_tt_rank=144)
ranks = shapes.tt_ranks(tt_with_ranks)
initialization = t3f.random_tensor(shape, tt_rank=10)
estimated = t3f.get_variable('estimated', initializer=initialization)
# Loss is MSE between the estimated and ground-truth tensor as computed in the observed cells.
loss = 0.5 * tf.reduce_sum((sparsity_mask * t3f.full(estimated) - sparse_observation)**2)
# Test loss is MSE between the estimated tensor and full (and not noisy) ground-truth tensor A.
test_loss = 0.5 * tf.reduce_sum((t3f.full(estimated) - ground_truth)**2)
rel_error1 = relative_error1(t3f.full(estimated), ground_truth)

# SGD optimization
The simplest way to solve the optimization problem is Stochastic Gradient Descent: let TensorFlow differentiate the loss w.r.t. the factors (cores) of the TensorTrain decomposition of the estimated tensor and minimize the loss with your favourite SGD variation.

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01, epsilon=1e-18)
step = optimizer.minimize(loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
train_loss_hist = []
test_loss_hist = []
for i in range(10000):
    _, tr_loss_v, test_loss_v, rel_error1_v, ranks_v = sess.run([step, loss, test_loss,rel_error1, ranks])
    train_loss_hist.append(tr_loss_v)
    test_loss_hist.append(test_loss_v)
    print(i, tr_loss_v, test_loss_v, rel_error1_v, ranks_v)
    #if i % 1000 == 0:
     #   print(i, tr_loss_v, test_loss_v, rel_error1_v)

(0, 1684039800000000.0, 2805402300000000.0, 1.0000004, array([  1,  53, 144, 144,   1], dtype=int32))
(1, 1684039400000000.0, 2805401800000000.0, 1.0000002, array([  1,  53, 144, 144,   1], dtype=int32))
(2, 1684039100000000.0, 2805401200000000.0, 1.0000001, array([  1,  53, 144, 144,   1], dtype=int32))
(3, 1684039100000000.0, 2805401200000000.0, 1.0000001, array([  1,  53, 144, 144,   1], dtype=int32))
(4, 1684038600000000.0, 2805401200000000.0, 1.0000001, array([  1,  53, 144, 144,   1], dtype=int32))
(5, 1684038300000000.0, 2805400400000000.0, 1.0, array([  1,  53, 144, 144,   1], dtype=int32))
(6, 1684037600000000.0, 2805399600000000.0, 0.9999999, array([  1,  53, 144, 144,   1], dtype=int32))
(7, 1684036500000000.0, 2805398300000000.0, 0.9999996, array([  1,  53, 144, 144,   1], dtype=int32))
(8, 1684035300000000.0, 2805396000000000.0, 0.9999992, array([  1,  53, 144, 144,   1], dtype=int32))
(9, 1684033300000000.0, 2805393700000000.0, 0.9999988, array([  1,  53, 144, 144,   1], 

(80, 1455331800000000.0, 2424744500000000.0, 0.9296844, array([  1,  53, 144, 144,   1], dtype=int32))
(81, 1440020300000000.0, 2399231300000000.0, 0.9247804, array([  1,  53, 144, 144,   1], dtype=int32))
(82, 1424007100000000.0, 2372514400000000.0, 0.919617, array([  1,  53, 144, 144,   1], dtype=int32))
(83, 1407278600000000.0, 2344570600000000.0, 0.9141853, array([  1,  53, 144, 144,   1], dtype=int32))
(84, 1389813300000000.0, 2315431600000000.0, 0.90848666, array([  1,  53, 144, 144,   1], dtype=int32))
(85, 1371634600000000.0, 2285096300000000.0, 0.9025158, array([  1,  53, 144, 144,   1], dtype=int32))
(86, 1352731400000000.0, 2253539300000000.0, 0.8962623, array([  1,  53, 144, 144,   1], dtype=int32))
(87, 1333123200000000.0, 2220782400000000.0, 0.8897245, array([  1,  53, 144, 144,   1], dtype=int32))
(88, 1312785300000000.0, 2186825000000000.0, 0.88289607, array([  1,  53, 144, 144,   1], dtype=int32))
(89, 1291773400000000.0, 2151676200000000.0, 0.87577194, array([  1,  53

(160, 175190140000000.0, 291915840000000.0, 0.32257554, array([  1,  53, 144, 144,   1], dtype=int32))
(161, 174742570000000.0, 291169930000000.0, 0.3221631, array([  1,  53, 144, 144,   1], dtype=int32))
(162, 174348730000000.0, 290514200000000.0, 0.32180014, array([  1,  53, 144, 144,   1], dtype=int32))
(163, 173999680000000.0, 289932570000000.0, 0.32147786, array([  1,  53, 144, 144,   1], dtype=int32))
(164, 173687300000000.0, 289409630000000.0, 0.3211878, array([  1,  53, 144, 144,   1], dtype=int32))
(165, 173405610000000.0, 288939930000000.0, 0.32092708, array([  1,  53, 144, 144,   1], dtype=int32))
(166, 173149300000000.0, 288513560000000.0, 0.32069018, array([  1,  53, 144, 144,   1], dtype=int32))
(167, 172913860000000.0, 288119960000000.0, 0.32047138, array([  1,  53, 144, 144,   1], dtype=int32))
(168, 172694500000000.0, 287756840000000.0, 0.32026935, array([  1,  53, 144, 144,   1], dtype=int32))
(169, 172490680000000.0, 287419100000000.0, 0.32008135, array([  1,  53, 14

(240, 168278950000000.0, 280409980000000.0, 0.31615445, array([  1,  53, 144, 144,   1], dtype=int32))
(241, 168234120000000.0, 280335250000000.0, 0.3161123, array([  1,  53, 144, 144,   1], dtype=int32))
(242, 168188340000000.0, 280259620000000.0, 0.3160697, array([  1,  53, 144, 144,   1], dtype=int32))
(243, 168142060000000.0, 280182440000000.0, 0.31602615, array([  1,  53, 144, 144,   1], dtype=int32))
(244, 168094990000000.0, 280103940000000.0, 0.3159819, array([  1,  53, 144, 144,   1], dtype=int32))
(245, 168047300000000.0, 280024300000000.0, 0.31593695, array([  1,  53, 144, 144,   1], dtype=int32))
(246, 167998280000000.0, 279943200000000.0, 0.31589118, array([  1,  53, 144, 144,   1], dtype=int32))
(247, 167948760000000.0, 279861010000000.0, 0.3158448, array([  1,  53, 144, 144,   1], dtype=int32))
(248, 167898140000000.0, 279777240000000.0, 0.31579754, array([  1,  53, 144, 144,   1], dtype=int32))
(249, 167846670000000.0, 279691160000000.0, 0.31574896, array([  1,  53, 144,

(320, 160454640000000.0, 267376580000000.0, 0.30871964, array([  1,  53, 144, 144,   1], dtype=int32))
(321, 160276890000000.0, 267080900000000.0, 0.30854887, array([  1,  53, 144, 144,   1], dtype=int32))
(322, 160097120000000.0, 266781400000000.0, 0.30837584, array([  1,  53, 144, 144,   1], dtype=int32))
(323, 159914480000000.0, 266477870000000.0, 0.30820036, array([  1,  53, 144, 144,   1], dtype=int32))
(324, 159728880000000.0, 266169240000000.0, 0.30802184, array([  1,  53, 144, 144,   1], dtype=int32))
(325, 159540890000000.0, 265856230000000.0, 0.30784068, array([  1,  53, 144, 144,   1], dtype=int32))
(326, 159350300000000.0, 265537750000000.0, 0.30765623, array([  1,  53, 144, 144,   1], dtype=int32))
(327, 159156800000000.0, 265215460000000.0, 0.30746946, array([  1,  53, 144, 144,   1], dtype=int32))
(328, 158961100000000.0, 264889240000000.0, 0.3072803, array([  1,  53, 144, 144,   1], dtype=int32))
(329, 158762740000000.0, 264558640000000.0, 0.3070885, array([  1,  53, 14

...

(8420, 1553227000000.0, 2588289000000.0, 0.030374488, array([  1,  53, 144, 144,   1], dtype=int32))
(8421, 1553220600000.0, 2588283500000.0, 0.030374454, array([  1,  53, 144, 144,   1], dtype=int32))
(8422, 1553217900000.0, 2588275400000.0, 0.030374408, array([  1,  53, 144, 144,   1], dtype=int32))
(8423, 1553213800000.0, 2588270400000.0, 0.030374378, array([  1,  53, 144, 144,   1], dtype=int32))
(8424, 1553210700000.0, 2588261000000.0, 0.03037432, array([  1,  53, 144, 144,   1], dtype=int32))
(8425, 1553206900000.0, 2588258900000.0, 0.03037431, array([  1,  53, 144, 144,   1], dtype=int32))
(8426, 1553203700000.0, 2588249000000.0, 0.030374251, array([  1,  53, 144, 144,   1], dtype=int32))
(8427, 1553198500000.0, 2588246300000.0, 0.030374236, array([  1,  53, 144, 144,   1], dtype=int32))
(8428, 1553195100000.0, 2588236300000.0, 0.030374177, array([  1,  53, 144, 144,   1], dtype=int32))
(8429, 1553189600000.0, 2588230000000.0, 0.030374141, array([  1,  53, 144, 144,   1], dtype=

(8502, 1552845400000.0, 2587668000000.0, 0.030370843, array([  1,  53, 144, 144,   1], dtype=int32))
(8503, 1552840100000.0, 2587661000000.0, 0.0303708, array([  1,  53, 144, 144,   1], dtype=int32))
(8504, 1552836000000.0, 2587653300000.0, 0.030370757, array([  1,  53, 144, 144,   1], dtype=int32))
(8505, 1552830000000.0, 2587646200000.0, 0.030370716, array([  1,  53, 144, 144,   1], dtype=int32))
(8506, 1552826100000.0, 2587637800000.0, 0.030370666, array([  1,  53, 144, 144,   1], dtype=int32))
(8507, 1552820200000.0, 2587629500000.0, 0.030370615, array([  1,  53, 144, 144,   1], dtype=int32))
(8508, 1552815800000.0, 2587621000000.0, 0.030370565, array([  1,  53, 144, 144,   1], dtype=int32))
(8509, 1552811400000.0, 2587614200000.0, 0.030370526, array([  1,  53, 144, 144,   1], dtype=int32))
(8510, 1552806000000.0, 2587605300000.0, 0.030370476, array([  1,  53, 144, 144,   1], dtype=int32))
(8511, 1552802600000.0, 2587599600000.0, 0.030370442, array([  1,  53, 144, 144,   1], dtype=

(8584, 1552472300000.0, 2587065000000.0, 0.030367306, array([  1,  53, 144, 144,   1], dtype=int32))
(8585, 1552469100000.0, 2587060900000.0, 0.03036728, array([  1,  53, 144, 144,   1], dtype=int32))
(8586, 1552462400000.0, 2587051200000.0, 0.030367222, array([  1,  53, 144, 144,   1], dtype=int32))
(8587, 1552459400000.0, 2587047200000.0, 0.0303672, array([  1,  53, 144, 144,   1], dtype=int32))
(8588, 1552454900000.0, 2587037000000.0, 0.030367138, array([  1,  53, 144, 144,   1], dtype=int32))
(8589, 1552450600000.0, 2587030200000.0, 0.030367099, array([  1,  53, 144, 144,   1], dtype=int32))
(8590, 1552447200000.0, 2587023000000.0, 0.030367058, array([  1,  53, 144, 144,   1], dtype=int32))
(8591, 1552442700000.0, 2587016600000.0, 0.030367019, array([  1,  53, 144, 144,   1], dtype=int32))
(8592, 1552437600000.0, 2587007400000.0, 0.030366965, array([  1,  53, 144, 144,   1], dtype=int32))
(8593, 1552432600000.0, 2587002700000.0, 0.030366939, array([  1,  53, 144, 144,   1], dtype=i

(8666, 1552118600000.0, 2586501200000.0, 0.030363994, array([  1,  53, 144, 144,   1], dtype=int32))
(8667, 1552113900000.0, 2586493000000.0, 0.030363947, array([  1,  53, 144, 144,   1], dtype=int32))
(8668, 1552109900000.0, 2586486800000.0, 0.030363912, array([  1,  53, 144, 144,   1], dtype=int32))
(8669, 1552106300000.0, 2586481000000.0, 0.030363878, array([  1,  53, 144, 144,   1], dtype=int32))
(8670, 1552102300000.0, 2586475200000.0, 0.030363841, array([  1,  53, 144, 144,   1], dtype=int32))
(8671, 1552098000000.0, 2586466800000.0, 0.030363794, array([  1,  53, 144, 144,   1], dtype=int32))
(8672, 1552093700000.0, 2586458000000.0, 0.03036374, array([  1,  53, 144, 144,   1], dtype=int32))
(8673, 1552089900000.0, 2586451600000.0, 0.030363703, array([  1,  53, 144, 144,   1], dtype=int32))
(8674, 1552085000000.0, 2586444600000.0, 0.030363664, array([  1,  53, 144, 144,   1], dtype=int32))
(8675, 1552082900000.0, 2586438800000.0, 0.030363627, array([  1,  53, 144, 144,   1], dtype

(8748, 1551775200000.0, 2585942600000.0, 0.030360717, array([  1,  53, 144, 144,   1], dtype=int32))
(8749, 1551773200000.0, 2585937600000.0, 0.030360688, array([  1,  53, 144, 144,   1], dtype=int32))
(8750, 1551766800000.0, 2585929200000.0, 0.030360637, array([  1,  53, 144, 144,   1], dtype=int32))
(8751, 1551765000000.0, 2585924700000.0, 0.03036061, array([  1,  53, 144, 144,   1], dtype=int32))
(8752, 1551759000000.0, 2585918200000.0, 0.030360574, array([  1,  53, 144, 144,   1], dtype=int32))
(8753, 1551755400000.0, 2585910300000.0, 0.030360527, array([  1,  53, 144, 144,   1], dtype=int32))
(8754, 1551750900000.0, 2585904300000.0, 0.03036049, array([  1,  53, 144, 144,   1], dtype=int32))
(8755, 1551746500000.0, 2585896000000.0, 0.030360444, array([  1,  53, 144, 144,   1], dtype=int32))
(8756, 1551742400000.0, 2585891700000.0, 0.030360417, array([  1,  53, 144, 144,   1], dtype=int32))
(8757, 1551738000000.0, 2585883300000.0, 0.030360367, array([  1,  53, 144, 144,   1], dtype=

In [ ]:
plt.loglog(train_loss_hist, label='train')
plt.loglog(test_loss_hist, label='test')
plt.xlabel('Iteration')
plt.ylabel('MSE Loss value')
plt.title('SGD completion')
plt.legend()


In [ ]:
#ground_truth_var = t3f.get_variable('ground_truth', initializer=ground_truth, reuse=True)

In [ ]:
#ground_truth.read_value()
ground_truth_val = ground_truth.eval(session=sess)

In [ ]:
estimated_val = sess.run(t3f.full(estimated))

In [ ]:
def relative_error(x_hat,x_true):
    percent_error = np.linalg.norm(x_hat - x_true) / np.linalg.norm(x_true)
    return percent_error

In [ ]:
rel_error = relative_error(estimated_val,ground_truth_val)

In [ ]:
rel_error

In [ ]:
#ten_ones = np.ones_like(mask)
#x_reconstr = mt.reconstruct(x_hat,x_true, ten_ones, mask)

In [ ]:
estimated_val.shape

In [ ]:
x_hat_img = mt.reconstruct_image_affine(x_true_org, estimated_val)


In [ ]:
from nilearn import plotting

In [ ]:
recovered_image = plotting.plot_epi(x_hat_img, bg_img=None,black_bg=True, cmap='jet', cut_coords=None) 

In [ ]:
print(x_hat_img)

In [ ]:
ground_truth_val.shape

In [ ]:
x = sess.run(tt_with_ranks)